![TF](https://www.gstatic.com/devrel-devsite/prod/ve312520032ba2ac0c4d23f7b46fc670cbbe051886a2d1f04563a5e4768ad9787/tensorflow/images/lockup.svg)

In [ ]:
import tensorflow as tf
import os, sys
tf.__version__

# Install the TF Object Detection API
Important here that we install the same version of the API as the TensorFlow version.

In [ ]:
%%capture
!pip install ../input/tf-model-garden-official-models/tf_models_official-2.3.0-py2.py3-none-any.whl --user
!pip install pycocotools

Next we copy the training script from the source code.

In [ ]:
!cp /kaggle/input/tf-model-garden-official-models/tf-models-official-2.3.0/tf-models-official-2.3.0/official/vision/detection/main.py .

"main.py" will create the file "params.yaml" with the model configuration in the model directory we specify. We need to add the training and evaluation files (which were created in [this notebook](https://www.kaggle.com/mistag/data-create-tfrecords-of-vinbigdata-chest-x-rays)), and also override some of the default parameters. This is done by creating another .yaml file with our custom settings:

In [ ]:
%%writefile ./config.yaml
architecture:
  num_classes: 14
train:
  train_file_pattern: 
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?0-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?2-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?3-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?4-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?5-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?6-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?7-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?8-of-020.tfrecord"
  total_steps: 18000
  batch_size: 12
  input_sharding: true
  iterations_per_loop: 1000
eval:
  eval_file_pattern: 
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?1-of-020.tfrecord"
    - "/kaggle/input/data-create-tfrecords-of-vinbigdata-chest-x-rays/VinBig-0?9-of-020.tfrecord"
  batch_size: 1
  eval_samples: 879
    
retinanet_parser:
  aug_rand_hflip: false
  aug_scale_max: 1.0
  aug_scale_min: 1.0

One important detail to be aware of is that the check-points created during training are about 260MByte each - and the maximum notebook diskspace use is 5GB. We could move the training from /kaggle/working to /tmp which has more space. Or we can increase the interval between checkpoints by setting the "iterations_per_loop" to desired interval. The default setting is 100 (steps). We increase it to 1000 in the .yaml file above. 

# Training
Running "main.py" below will perform training. The output is captured to file for parsing further down.

In [ ]:
%%capture cap
!python main.py \
  --strategy_type=one_device \
  --num_gpus=1 \
  --model_dir=./model1 \
  --mode=train \
  --model=retinanet \
  --config_file="./config.yaml"

In [ ]:
with open('./train.log', 'w') as f:
    f.write(cap.stdout)

Check the learning curves (by parsing the log file).

In [ ]:
%%capture
!pip install --user parse

In [ ]:
from parse import *
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with open('./train.log', 'r') as f:
    data=f.read()
loss=[]
for r in findall("\'model_loss\': {:f}", data):
    loss.append(r[0])
cls=[] 
for r in findall("\'cls_loss\': {:f}", data): # class loss
    cls.append(r[0])
step=[]
for r in findall("Train Step: {:d}", data):
    step.append(r[0])
plt.figure(figsize=(16, 8))
plt.plot(step,loss[0::2])
plt.plot(step,cls[0::2])
plt.legend(['model loss', 'class loss'])
plt.xlabel('Global step')
plt.title('Learning curves');

That's it! The trained model in the "model1" directory can now be used for predictions, but that must be done in a notebook without internet enabled for submission...  
Also remember to use the same preprocessing (CLAHE) on the test images as the [TFRecords creation notebook](https://www.kaggle.com/mistag/data-create-tfrecords-of-vinbigdata-chest-x-rays). The CLAHE parameters used can be found in a .json file:

In [ ]:
import json 

with open('../input/data-create-tfrecords-of-vinbigdata-chest-x-rays/dparams.json') as json_file:
     dparams = json.load(json_file)
dparams